# 連立常微分方程式
連立常微分方程式を解く場合も1次元の場合と同じ．  
## 例：ローレンツ方程式  
<br>
<img src="https://latex.codecogs.com/png.image?\dpi{120}&space;\bg_white&space;\begin{pmatrix}\dot{x}\\&space;\dot{y}\\&space;\dot{z}\end{pmatrix}=\begin{pmatrix}-px&plus;py\\-xz&plus;rx-y\\&space;xy-bz\\\end{pmatrix}" title="\bg_white \begin{pmatrix}\dot{x}\\ \dot{y}\\ \dot{z}\end{pmatrix}=\begin{pmatrix}-px+py\\-xz+rx-y\\ xy-bz\\\end{pmatrix}" />  
<br>


In [3]:
using DifferentialEquations
using ParameterizedFunctions

ここでは`@ode_def`マクロを使う  
### 微分方程式の定義

In [4]:
lorenz_m = @ode_def begin
    dx = -p*x + p*y
    dy = -x*z + r*x -y
    dz = x*y - b*z
end p r b

(::var"##257"{var"###ParameterizedDiffEqFunction#259", var"###ParameterizedTGradFunction#260", var"###ParameterizedJacobianFunction#261", Nothing, Nothing, ODESystem}) (generic function with 1 method)

### (微分方程式の描画)
`Latexify`を使うと定義した微分方程式を綺麗に表示できる．  
微分方程式のチェック？などに使えるかも  

In [5]:
using Latexify
latexify(lorenz_m)

L"\begin{align}
\frac{dx(t)}{dt} =& x\left( t \right) \left(  - p \right) + p y\left( t \right) \\
\frac{dy(t)}{dt} =& z\left( t \right) \left(  - x\left( t \right) \right) + r x\left( t \right) - y\left( t \right) \\
\frac{dz(t)}{dt} =& x\left( t \right) y\left( t \right) - b z\left( t \right)
\end{align}
"

### 問題を定義
1次元のときと同じ

In [6]:
x₀ = [0.0, 4.0, 28.0]
p = 10
r = 28
b = 8/3
t_span = (0, 50.0)

prob = ODEProblem(lorenz_m, x₀, t_span, (p, r, b))

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 50.0)
u0: 3-element Vector{Float64}:
  0.0
  4.0
 28.0

### 解く

In [7]:
sol = solve(prob);

### 解のプロット

In [8]:
using Plots

In [9]:
plot(sol)
savefig("sb2_1.png")

![pic](picture/sb2_1.png)

### 解について
解は`ODECompositeSolution`として返される．  
`sol[i]`でi番目の解にアクセスできる．  

In [10]:
sol[2]

3-element Vector{Float64}:
  0.0014137366927165365
  3.999858601385078
 27.997360736072547

`sol.t` : 時刻歴の配列

In [11]:
sol.t[1:5]

5-element Vector{Float64}:
 0.0
 3.535028953698351e-5
 0.0003888531849068186
 0.003923882138605169
 0.020333768098013315

`sol.u` : 解ベクトルの配列

In [12]:
sol.u[1:5]

5-element Vector{Vector{Float64}}:
 [0.0, 4.0, 28.0]
 [0.0014137366927165365, 3.999858601385078, 27.997360736072547]
 [0.015520905424812406, 3.998444948030281, 27.970992749143974]
 [0.15361282439337523, 3.98439393599714, 27.709751122448132]
 [0.7286763765799963, 3.9268855053350147, 26.551972589469933]

`sol.prob` : 初期値問題の設定内容

In [13]:
sol.prob

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 50.0)
u0: 3-element Vector{Float64}:
  0.0
  4.0
 28.0

他にも色々なデータが入ってます．  
おもしろい機能として解の補完があります．  
```julia
sol(時間)
```
で`t=時間`での補完した解が得られます．  
`[]`ではなく`()`なことに注意（つまりこれは関数．恐らく） 

In [14]:
sol(1.2345)

3-element Vector{Float64}:
 11.736388307019514
  6.081725428276207
 36.572003681705

この機能があるため`scipy.integrate.solve_ivp`と異なり，刻み時間を指定しなくても初めからプロットがなめらかです．  
せっかくなので解を3次元でプロットしてみます．

In [15]:
plot(sol,vars=(1,2,3))
savefig("sb2_2.png")

![pic](picture/sb2_2.png)

****
## 高速化
微分方程式の定義方法によって計算速度はかなり変動します．  
公式wiki : [https://diffeq.sciml.ai/stable/tutorials/faster_ode_example/#Code-Optimization-in-Julia](https://diffeq.sciml.ai/stable/tutorials/faster_ode_example/#Code-Optimization-in-Julia)

In [21]:
"""ローレンツ方程式"""
function lorenz(X, param, t)
    x, y, z = X
    p, r, b = param

    dx = -p*x + p*y
    dy = -x*z + r*x -y
    dz = x*y - b*z
    
    [dx, dy, dz]  # ベクトルを返す
end

prob2 = ODEProblem(lorenz, x₀, t_span, (p, r, b))

上のように速度をベクトルで返しても解けますが，配列に書き出すほうがかなり高速です  

In [22]:
"""ローレンツ方程式．配列に書き出す方式

juliaでは引数を破壊するような関数には!をつける
"""
function lorenz!(dX, X, param, t)
    x, y, z = X
    p, r, b = param

    dX[1] = -p*x + p*y
    dX[2] = -x*z + r*x -y
    dX[3] = x*y - b*z
    
    nothing  # 何も返さない
end

prob3 = ODEProblem(lorenz!, x₀, t_span, (p, r, b))

### ベンチマーク
BenchmarkTools.jlを使って比較

In [25]:
using BenchmarkTools

配列を返す：`lonrenz`

In [33]:
@benchmark solve(prob2)  # 書き出さない

BenchmarkTools.Trial: 1597 samples with 1 evaluation.
 Range (min … max):  1.925 ms … 12.304 ms  ┊ GC (min … max):  0.00% … 68.69%
 Time  (median):     2.259 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.110 ms ±  1.914 ms  ┊ GC (mean ± σ):  12.99% ± 16.15%

  █▆▄▃▃ ▁▁ ▂▃▅▃▃▂▁                                            
  ████████▇████████▇▇▄▆▁▄▄▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▄▅▄▆▆▆▇▇▇▆▇▇▆ █
  1.93 ms      Histogram: log(frequency) by time     10.6 ms <

 Memory estimate: 6.06 MiB, allocs estimate: 56056.

配列書き出し：`lorenz!`

In [34]:
@benchmark solve(prob3)  # 書き出し

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  334.100 μs …   9.514 ms  ┊ GC (min … max): 0.00% … 94.78%
 Time  (median):     357.350 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   473.382 μs ± 576.173 μs  ┊ GC (mean ± σ):  9.87% ±  7.77%

  █▇▅▃▃▄▃▂▁▂▁▁▁▁▁▃▂▂▆▅▃▁▁▂▂ ▁                                   ▂
  █████████████████████████████▇██▇▇▇▇▇▇▇▇▆▆▆▆▅▅▄▅▅▅▄▄▄▅▅▄▄▄▄▄▄ █
  334 μs        Histogram: log(frequency) by time        979 μs <

 Memory estimate: 747.67 KiB, allocs estimate: 6188.

`lorenz!`のほうがメモリ使用量も少なく，かなり高速なことがわかる．  
<br>
`@ode_def`マクロを使って微分方程式を定義すると，自動的に配列書き出しの関数定義になる．  

In [35]:
@benchmark solve(prob)  # @ode_def

BenchmarkTools.Trial: 9789 samples with 1 evaluation.
 Range (min … max):  345.000 μs …   9.974 ms  ┊ GC (min … max):  0.00% … 95.02%
 Time  (median):     390.100 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   504.549 μs ± 626.650 μs  ┊ GC (mean ± σ):  10.05% ±  7.80%

  █▇▄▄▄▄▃▃▃▂▂▂▂▃▃▂▅▅▃▂▂▂▂▁▁▁▁▁                                  ▂
  █████████████████████████████████▇█▇█▆▆▇▆▆▄▅▆▅▆▅▄▅▄▅▄▃▅▄▄▄▄▄▃ █
  345 μs        Histogram: log(frequency) by time       1.09 ms <

 Memory estimate: 749.52 KiB, allocs estimate: 6197.

配列書き出しとほぼ同じメモリ使用量，計算時間なことがわかる